## Database Connection

In [1]:
import sqlite3
import pandas as pd
import numpy as np
from difflib import SequenceMatcher

import igraph
import sys
import tweepy
import networkx as nx
import pandas as pd
import numpy as np
import json
import csv
import ast
from operator import itemgetter
import re
from karateclub import Graph2Vec
import xgboost as xgb
import matplotlib.pyplot as plt
import os

In [ ]:
access_token = "1404724320451153923-Itg2VG0Hws5MeNzLbYuzewGmYuK9Vc"
access_token_secret = "73hFnqmeiVtYQtnhZxF4Z9Mz1bz9e3IaPDSY2oRnOr6HA"

consumer_key = "LeFaQFHWCgEL2IbFKmZ5O8Qb1"
consumer_secret = "HjHWBEZCszmum3s84EEAYd5tkzAUeugBzMXzTasAMgSR580Q7B"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [2]:
pd.set_option('display.max_colwidth', None)  
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [3]:
conn_tweets = sqlite3.connect('tweets_airlines_clean.db')

## User categorization

The cells below categorize users based on certain conditions

In [4]:
query = '''SELECT *
           FROM TWEETS'''

df_tweets = pd.read_sql(query, conn_tweets)

### Airline detection

In [5]:
airline_id = {56377143: 'KLM', 106062176: 'AirFrance', 18332190: 'British_Airways', 22536055: 'AmericanAir', 
              124476322: 'Lufthansa', 26223583: 'AirBerlin', 2182373406: 'AirBerlin assist',
              38676903: 'easyJet', 1542862735: 'RyanAir', 253340062: 'SingaporeAir', 
              218730857: 'Qantas', 45621423: 'EtihadAirways', 20626359: 'VirginAtlantic'}

In [6]:
def is_airline(user_id: int):
    category = airline_id.get(user_id, 'user')
    
    if category == 'user':
        return category
    else:
        return 'airline ' + category


In [7]:
df_tweets['user_category'] = df_tweets['user_id'].apply(lambda x: is_airline(x))

### Bots detection

##### Method 1: Accounts with Tweet output that is at least 70% similar

In [8]:
normal_users = df_tweets['user_category'] == 'user'

In [9]:
df_findbots = pd.DataFrame(df_tweets[normal_users].groupby('user_id')['full_text'].apply(lambda g: g.values.tolist()))
df_findbots.reset_index(inplace=True)

In [21]:
subset_findbots = df_findbots.head(10000).copy()

In [26]:
def is_bot(tweets: list):
    if len(tweets) > 3:
        ratios = [SequenceMatcher(None, tweet, other).ratio() for i, tweet in enumerate(tweets) for other in tweets[i+1:]]

        mean_match = sum(ratios) / len(ratios)

        if mean_match > 0.7:
            return 'bot'
        else:
            return 'user'
    else: 
        return 'user'


In [23]:
subset_findbots['user_category'] = subset_findbots['full_text'].apply(lambda tweets: is_bot(tweets))

In [24]:
bots = subset_findbots['user_category'] == 'bot'

In [25]:
subset_findbots[bots]

,user_id,full_text,user_category
9743,9279052,"[Qantas Joins Growing Apple MacBook Ban... https://t.co/EFdXtCsUbL, Random Dumber Generator: Qantas Joins Growing Apple MacBook Ban https://t.co/nLFu4TUIDQ, Qantas Joins Growing Apple MacBook Ban https://t.co/gGgt8QMM5M, Random Dumber Generator: Qantas Joins Growing Apple MacBook Ban ... https://t.co/nLFu4TUIDQ, Random Dumber Generator: Qantas Joins Growing Apple MacBook Ban... https://t.co/nLFu4TUIDQ, randomdumber: Qantas Joins Growing Apple MacBook Ban https://t.co/5OtvVOm4te, Random Dumber Generator: Qantas Joins Growing Apple MacBook Ban - https://t.co/nLFu4TUIDQ, Random Dumber Generator: Qantas Joins Growing Apple MacBook Ban -- https://t.co/nLFu4TUIDQ, Qantas Joins Growing Apple MacBook Ban https://t.co/EFdXtCsUbL]",bot


In [279]:
bots = list(df_findbots[df_findbots['user_category'] == 'bot']['user_id'])

In [283]:
df_tweets.loc[df_tweets['user_id'].isin(bots), 'user_category'] = 'bot'

##### Method 2: Accounts with multiple Tweets that are posted within 5 seconds after eachother

In [47]:
q_time = '''SELECT *
            FROM TWEETS
            LIMIT 100000'''

df_time = pd.read_sql(q_time, conn_tweets)

In [96]:
df_time = df_time.groupby(['user_id'])

In [97]:
df_time.sort_values(by='user_id', inplace=True)

In [100]:
possibly_botty = df_time['difference'] < 5

In [101]:
df_time[possibly_botty].tail(50)

,user_id,tweet_id,full_text,created_at,reply_to_tweet,reply_to_user,difference
60881,1121377872386691077,1133054333942206465,"@KLM Dus ik moet elke dag patat eten zodat KLM vluchten op 3400 meter met een ""soort van betere brandstof"" over National Park Weerribben Wieden de landing in kunnen gaan zetten @RedWieden @SchipholWatch @Steenwijkerland . https://t.co/e9acApxd8M",1558976179000000000,0,56377143,-597303000000000.00000
44250,1121377872386691077,1135559608054009858,@jaapmodder @Dunja2004 @Schiphol @KLM Over Nationaal Park @RedWieden Weerribben steeds vaker vluchten onder de 4000 meter. #luchtvaartbubble. Ik heb ook gehoord dat als een toestel enige vertraging heeft ze sneller vliegen en dus meer lawaai produceren. https://t.co/Sx0ZGbirKW,1559573482000000000,1134903787787816960,145948716,-108602000000000.00000
98067,1121461965598019584,1133826550409785344,"RT @JasonUnsworth2: The revival of @jetairways is important for many, from its employees to its loyal customers, to Sir Naresh Goyal himsel…",1559160289000000000,0,0,-411804000000000.00000
15204,1121461965598019584,1132165913682145280,RT @jayavel_dk: #JetAirways #JetAirwaysCrisis #9W #HELP #SaveJetAirways #SaveLives #Aviation #Airline\n\n@narendramodi @TheOfficialSBI @Etiha…,1558764363000000000,0,0,-884421000000000.00000
45211,1121714203906256896,1135571194277715969,"Ryanair moet reizigers schadevergoeding geven voor pilotenstaking'May 30, 2019 at 02:44PM @ImPblq",1559576245000000000,0,0,-82494000000000.00000
49396,1121736012106031106,1135629416606961664,RT @Ryanair_IT: Buon lunedì da @BLQairport 🌞 🇮🇹\n\nSiete in partenza? In questa sezione del sito potrete controllare lo stato di tutti i nost…,1559590126000000000,0,0,-267434000000000.00000
70907,1121856339297415174,1131302322745806850,"Ryanair Boeing 737NG 8AS/W (EI-EVL, #4CA9D4) as flight #RYR97 at 40000 ft heading north west bound OTT north east of #RyanHQ",1558558466000000000,0,0,0.00000
64838,1122620459143634945,1131176762438758400,Ryanair Delays Boeing 737 MAX Deliveries Following Grounding en Ryanair https://t.co/4E5Kc8F7EE,1558528531000000000,0,0,-329472000000000.00000
56040,1122887851299999744,1132936054619738112,"Thomas Cook, easyJet &amp; TUI: How to protect flights and holidays amid ‘turbulent’ times https://t.co/jHXzZaZvGP",1558947979000000000,0,0,-938900000000000.00000
67033,1122887851299999744,1131223836391870464,Scotland airport strike: Flights including easyJet cancelled at Inverness – is yours? https://t.co/sGVLdNv3nI https://t.co/sIC8WZJDur,1558539754000000000,0,0,-422934000000000.00000


1152

In [ ]:
class TweetGrabber():
	
	def __init__(self,myApi,sApi,at,sAt):
		self.tweepy = tweepy
		auth = tweepy.OAuthHandler(myApi, sApi)
		auth.set_access_token(at, sAt)
		self.api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
		
	#Return the string without non ASCII characters
	def strip_non_ascii(self,string):
		
		stripped = (c for c in string if 0 < ord(c) < 127)
		return ''.join(stripped)  
		
	def user_search(self,user,csv_prefix):
		API_results = self.tweepy.Cursor(self.api.user_timeline,id=user,tweet_mode='extended').items()

		with open(f'{csv_prefix}.csv', 'w', newline='') as csvfile:
			fieldnames = ['tweet_id', 'tweet_text', 'date', 'user_id', 'user_mentions', 'retweet_count']
			writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
			writer.writeheader()

			n = 0
			while n <= 25:
				for tweet in API_results:
					n += 1
					try:               
						text = self.strip_non_ascii(tweet.full_text)
						date = tweet.created_at.strftime('%m/%d/%Y')        
						writer.writerow({
										'tweet_id': tweet.id_str,
										'tweet_text': text,
										'date': date,
										'user_id': tweet.user.id_str,
										'user_mentions':tweet.entities['user_mentions'],
										'retweet_count': tweet.retweet_count
										})
					except:
						continue

# Process the created CSV in order to generate edge list
class RetweetParser():
	
	def __init__(self,data,user):
		self.user = user

		edge_list = []
	
		for idx,row in data.iterrows():
			if len(row[4]) > 5:    
				user_account = user
				weight = np.log(row[5] + 1)
				for idx_1, item in enumerate(ast.literal_eval(row[4])):
					edge_list.append((user_account,item['screen_name'],weight))

					for idx_2 in range(idx_1+1,len(ast.literal_eval(row[4]))):
						name_a = ast.literal_eval(row[4])[idx_1]['screen_name']
						name_b = ast.literal_eval(row[4])[idx_2]['screen_name']

						edge_list.append((name_a,name_b,weight))
		
		with open(f'{self.user}.csv', 'w', newline='') as csvfile:
			fieldnames = ['user_a', 'user_b', 'log_retweet']
			writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
			writer.writeheader()

			for row in edge_list:        
				writer.writerow({
								'user_a': row[0],
								'user_b': row[1],
								'log_retweet': row[2]
								})


# Eigenvector centrality measures 'influence' of each node within the graph network
class TweetGraph():
	def __init__(self,edge_list):
		data = pd.read_csv(edge_list).to_records(index=False)
		self.tuple_graph = igraph.Graph.TupleList(data, weights=True, directed=False)
		
	def e_centrality(self):
		vectors = self.tuple_graph.eigenvector_centrality()
		e = {name:cen for cen, name in  zip([v for v in vectors],self.tuple_graph.vs['name'])}
		return sorted(e.items(), key=itemgetter(1),reverse=True)

In [ ]:
lst_users = list(set(df_tweets['screen_name']))
lst1 = lst_users[:500000]
lst2 = lst_users[500000:1000000]
lst3 = lst_users[1000000:1500000]
lst4 = lst_users[150000:2000000]
lst5 = lst_users[2000000:]

In [ ]:
i = 0
length = len(lst1)
for name in lst1:
    i += 1
    t = TweetGrabber(
        myApi = consumer_key,
        sApi = consumer_secret,
        at = access_token,
        sAt = access_token_secret)

    screen_name = name
    
    try:
        #Collect the user's mentions into a CSV titled with their username
        t.user_search(user=screen_name, csv_prefix=screen_name)

        #Read the created CSV into a pandas DataFrame for input to RetweetParser class
        userFrame = pd.read_csv(screen_name + ".csv", encoding="ISO-8859-1")

        #RetweetParser overwrites the first CSV with a weighted edgelist
        r = RetweetParser(userFrame, screen_name)

        #The weighted, undirected iGraph object
        log_graph = TweetGraph(edge_list= screen_name + ".csv")

        #Add 'size' attribute to each vertex based on its Eigencentrality
        #NOTE: multiplying the value by some consistent large number creates a more intuitive
        #plot, viewing-wise, but doesn't impact classification, since this change is applied
        #to all vertices
        for key, value in log_graph.e_centrality():
            log_graph.tuple_graph.vs.find(name=key)['size'] = value*20

        #Save the graph in GML format
        log_graph.tuple_graph.write_gml(f=screen_name+".gml")

        igraph_gml = open(screen_name+".gml", 'r')
        lof = igraph_gml.readlines()
        igraph_gml.close()
        if lof[4]!="multigraph 1":
          lof.insert(4, "multigraph 1\n")
        igraph_gml = open(screen_name + '.gml', 'w')
        lof = "".join(lof)
        igraph_gml.write(lof)
        igraph_gml.close()
        

        # Next, read the GML with NetworkX, then convert each
        # node from being labeled by name to being labeled by sequential
        # integers, since Graph2Vec requires nodes to be labeled this way
        H = nx.read_gml(screen_name + '.gml', label='name')
        convertedgraph = nx.convert_node_labels_to_integers(H)
    except:
        pass
        
    try:
        embedding_model = Graph2Vec(dimensions=64)
        embedding_model.fit([convertedgraph])
        embeddingsframe = pd.DataFrame(embedding_model.get_embedding())
    except:
        pass
    
    try:
        classification_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42, learning_rate = 0.05, n_estimators = 5000, early_stopping_rounds = 10)
        classification_model.load_model('graph_classifier_3.json')
        pred = classification_model.predict(embeddingsframe)
        classification_value = pred[0]  
    except:
        pass
    try:
        if classification_value == 'bot':
            df_tweets = df_tweets.drop(list(df_tweets[df_tweets['screen_name'] == screen_name].index))
    except:
        pass
    
    try:
        os.remove(screen_name + '.csv')
        os.remove(screen_name + '.gml')
    except:
        pass
    print(i/length)

In [7]:
conn_tweets.close()